In [1]:
import setup
setup.init_django()

In [2]:
from market import services as market_services
from market import tasks as market_tasks
from market.models import Company

In [3]:
import json
from decouple import config

In [4]:
ticker = "AAPL"
name = "Apple"
company, _ = Company.objects.get_or_create(name=name, ticker=ticker)
company.id

1

In [6]:
market_tasks.sync_historical_stock_data(years_ago=1, company_ids=[company.id], use_celery=False, verbose=True)

# use celery / async
# market_tasks.sync_historical_stock_data.delay(years_ago=5, company_ids=[company.id], use_celery=True, verbose=False)

Historical sync days ago 30
dataset length 4197
Doing chunk 0
finished chunk 0
Doing chunk 1000
finished chunk 1000
Doing chunk 2000
finished chunk 2000
Doing chunk 3000
finished chunk 3000
Doing chunk 4000
finished chunk 4000
30 done

Historical sync days ago 60
dataset length 8043
Doing chunk 0
finished chunk 0
Doing chunk 1000
finished chunk 1000
Doing chunk 2000
finished chunk 2000
Doing chunk 3000
finished chunk 3000
Doing chunk 4000
finished chunk 4000
Doing chunk 5000
finished chunk 5000
Doing chunk 6000
finished chunk 6000
Doing chunk 7000
finished chunk 7000
Doing chunk 8000
finished chunk 8000
60 done

Historical sync days ago 90
dataset length 11692
Doing chunk 0
finished chunk 0
Doing chunk 1000
finished chunk 1000
Doing chunk 2000
finished chunk 2000
Doing chunk 3000
finished chunk 3000
Doing chunk 4000
finished chunk 4000
Doing chunk 5000
finished chunk 5000
Doing chunk 6000
finished chunk 6000
Doing chunk 7000
finished chunk 7000
Doing chunk 8000
finished chunk 8000
Doin

In [18]:
results = market_services.get_stock_indicators(ticker=ticker, days=90)
results

{'score': -1,
 'ticker': 'AAPL',
 'indicators': {'ma_5': 225.598,
  'ma_20': 227.6609,
  'current_price': 227.75,
  'conservative_target': 235.5237,
  'aggressive_target': 240.3263,
  'average_price': 226.4995,
  'avg_volume': 4692.446153846154,
  'latest_volume': 341,
  'volume_change_percent': -92.7330013212725,
  'rsi': 48.8471,
  'avg_gain': 0.9153,
  'avg_loss': 0.9585,
  'period': 14,
  'days': 90}}

In [8]:
results_as_json = json.dumps(results)
results_as_json

'{"score": -1, "ticker": "AAPL", "indicators": {"ma_5": 225.598, "ma_20": 227.6609, "current_price": 227.75, "conservative_target": 233.4647, "aggressive_target": 236.9953, "average_price": 228.572, "avg_volume": 4489.916666666667, "latest_volume": 341, "volume_change_percent": -92.4052042539765, "rsi": 42.9317, "avg_gain": 0.8474, "avg_loss": 1.1265, "period": 14, "days": 30}}'

In [9]:
OPENAI_API_KEY=config("OPENAI_API_KEY", default=None)

In [10]:
assert OPENAI_API_KEY is not None

In [11]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

In [15]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are an expert an analyzing stocks and respond in JSON data"},
        {"role": "user", "content": f"Considering these results {results_as_json}, provide a recommendation"}
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "recommendation",
            "schema": {
                "type": "object",
                "properties": {
                    "buy": {
                        "description": "Recommend to buy stock",
                        "type": "boolean"
                    },
                    "sell": {
                        "description": "Recommend to sell stock",
                        "type": "boolean"
                    },
                    "hold": {
                        "description": "Recommend to hold stock",
                        "type": "boolean"
                    },
                    "explanation": {
                        "description": "Explanation of reasoning in 1 or 2 sentences",
                        "type": "string"
                    },
                    "additionalProperties": False
                }
            }
        }
    }
)

In [16]:
result = json.loads(response.choices[0].message.content)
result

{'buy': False,
 'sell': True,
 'hold': False,
 'explanation': 'Given the negative score and significant drop in volume along with an RSI of 42, which indicates potential weakness, it is advisable to sell the stock.'}

In [17]:
result.get('hold') is True

False